In [1]:
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification
import torch
import torch.nn as nn
from PIL import Image
import pathlib
from typing import *
import pandas as pd
import shutil
import os

processor = AutoProcessor.from_pretrained("fummicc1/hiyoshi-street-clip")
# processor = AutoProcessor.from_pretrained("geolocal/StreetCLIP")

model = AutoModelForZeroShotImageClassification.from_pretrained("fummicc1/hiyoshi-street-clip")
# model = AutoModelForZeroShotImageClassification.from_pretrained("geolocal/StreetCLIP")

/Users/fumiyatanaka/Work/Univ/Spr/MachineIntelligence/Machine-Intelligence-Team-8/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
town_name = "Hiyoshi"
predict_town = True

postal_code_maps = {
    'Minowacho': '223-0051',
    'HiyoshiHoncho': '223-0062',
    'Hiyoshi': '223-0061',
}

hiyoshi_maps = {
    "1-chome": "〒223-0061 神奈川県横浜市港北区日吉1丁目",
    "2-chome": "〒223-0061 神奈川県横浜市港北区日吉2丁目",
    "3-chome": "〒223-0061 神奈川県横浜市港北区日吉3丁目",
    "4-chome": "〒223-0061 神奈川県横浜市港北区日吉4丁目",
    "5-chome": "〒223-0061 神奈川県横浜市港北区日吉5丁目",
    "6-chome": "〒223-0061 神奈川県横浜市港北区日吉6丁目",
    "7-chome": "〒223-0061 神奈川県横浜市港北区日吉7丁目",
}
label_maps = {
    'Hiyoshi': 'Hiyoshi',
    'Hiyoshihoncho': 'Hiyoshihonchol',
    'Minowacho': 'Minowacho',
}
labels = []

if predict_town:
    labels = list(label_maps.values())
else:
    pass
    # for folder in sorted(list(town.iterdir())):
    #     name = folder.name
    #     # if maps.get(name) is not None:
    #         # labels.append(maps[name])
    #     if name not in hiyoshi_maps:
    #         continue
    #     labels.append(name)
print(labels)

['Hiyoshi', 'Hiyoshihonchol', 'Minowacho']


In [3]:
def predict(labels: List[str], img: torch.Tensor):
    inputs = processor(
        text=labels,
        images=img,
        return_tensors="pt",
        padding=True
    )
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1)
    print("probs:", probs)
    index = torch.argmax(probs, dim=1).item()
    return labels[index]

In [4]:

path = pathlib.Path(f"data/town/train/{town_name}")

acc = 0

paths = list(path.iterdir())[100:]

for img_path in paths:
    img = Image.open(img_path.as_posix()).convert("RGB").resize((336, 336))
    print("img:", img_path.name)
    ret = predict(labels, img=img)
    print("result", ret)
    if ret == label_maps[town_name]:
        acc += 1
acc /= len(paths)
acc

img: 3555180910-13965033900.png
probs: tensor([[0.2937, 0.3658, 0.3406]], grad_fn=<SoftmaxBackward0>)
result Hiyoshihonchol
img: 3555231734-13965084786.png
probs: tensor([[0.3914, 0.2814, 0.3271]], grad_fn=<SoftmaxBackward0>)
result Hiyoshi
img: 3555130085-13965644529.png
probs: tensor([[0.3790, 0.2929, 0.3281]], grad_fn=<SoftmaxBackward0>)
result Hiyoshi
img: 3555841627-13964830357.png
probs: tensor([[0.4291, 0.2527, 0.3182]], grad_fn=<SoftmaxBackward0>)
result Hiyoshi
img: 3555536680-13964677700.png
probs: tensor([[0.2871, 0.3732, 0.3398]], grad_fn=<SoftmaxBackward0>)
result Hiyoshihonchol
img: 3554469368-13964983014.png
probs: tensor([[0.3984, 0.2767, 0.3249]], grad_fn=<SoftmaxBackward0>)
result Hiyoshi
img: 3554672666-13965542757.png
probs: tensor([[0.2929, 0.3672, 0.3399]], grad_fn=<SoftmaxBackward0>)
result Hiyoshihonchol
img: 3555485856-13964983014.png
probs: tensor([[0.4037, 0.2726, 0.3237]], grad_fn=<SoftmaxBackward0>)
result Hiyoshi
img: 3555231734-13965237443.png
probs: tens

0.5694980694980695

# Minowacho

## ACCURACY

|pretrained|finetuning|
|---|---|
|0.0|0.06164383561643835|

## F1

|pretrained|finetuning|
|---|---|
|||

# Hiyoshi

## ACCURACY

|pretrained|finetuning|
|---|---|
||0.5694980694980695|

## F1

|pretrained|finetuning|
|---|---|
|||

# Hiyochihoncho

## ACCURACY

|pretrained|finetuning|
|---|---|
||0.34615384615384615|

## F1

|pretrained|finetuning|
|---|---|
|||